In [336]:
import numpy as np
import csv
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler


/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In this script, we have three ML model (using 5 fold cross validation):
SVC: C-Support Vector Classification.
SGDClassifier: Linear classifiers (SVM, logistic regression, a.o.) with SGD training.
This estimator implements regularized linear models with stochastic gradient descent (SGD) learning: the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing strength schedule (aka learning rate). SGD allows minibatch (online/out-of-core) learning, see the partial_fit method. For best results using the default learning rate schedule, the data should have zero mean and unit variance.

Random Forest:
RF gives the best result:
f1: 0.95 (+/- 0.08)
precision: 0.8666666666666668
recall: 0.8428571428571429

Rule based: F1: 0.98245614   Recall: 1.0 Precision: 0.8125


In [2]:
class myQuote:
    def __init__(self, text):
        #read in CSV, process
        self.quoteText = text
        self.quoteID = hash(self.quoteText)
        self.link = []
        self.name = []
        self.description = []
        self.scores = []
        self.cos = []
    
    def __hash__(self):
        return self.quoteID
    
    def __str__(self):
        return "Object text: " + self.quoteText + '\n' +\
        "Links: " + str(self.link) + '\n' +\
        "Names: " + str(self.name) + '\n' +\
        "Description " + str(self.description) + '\n' +\
        "Scores: " + str(self.scores)
        

In [495]:
infile = open('searchResults','rb')
results = pickle.load(infile, encoding='bytes')

In [602]:
labels = pd.read_csv('./LableResults/MLData_recode_mannual.csv')
label = labels['label'].values
label = label.astype(np.float)
print(len(label))
print(type(label))


200
<class 'numpy.ndarray'>


In [515]:
#get the first 200 labels: those are the one in the mannual annotate sample
def append_features(ob):
    count = 0
    proto_matrix = []
    for item in ob:
        col2 = np.append(ob[item].scores, ob[item].cos)
        #print(col2)
        proto_matrix.append(col2)
        count += 1
        if count > 199:
            break
            
proto_matrix = append_features(results)
FeatureMatrix = np.matrix(proto_matrix)

In [516]:
FeatureMatrix[0:10][0:3]

matrix([[1.        , 1.        , 5.        , 7.        , 5.        ,
         7.        , 1.        , 1.        , 6.        , 0.        ,
         0.90142585],
        [5.        , 2.        , 3.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 3.        , 0.        ,
         0.97699932],
        [5.        , 7.        , 3.        , 5.        , 9.        ,
         9.        , 5.        , 1.        , 1.        , 0.        ,
         0.96864645]])

In [517]:
scaler = StandardScaler()
scaled_matrix = scaler.fit_transform(FeatureMatrix)


In [633]:
#data
X = scaled_matrix
y = label

In [603]:
#split data for grish search
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [607]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
svc = svm.SVC()
parameters = [{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': [0.01, 0.001, 0.0001],
                     'C': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0, 3.0], 'class_weight':['balanced']}]
                   
grid_search_item = GridSearchCV(estimator = svc,
                          param_grid = parameters,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_train, y_train)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.9
{'C': 1.7, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'rbf'}
0.835714 (0.060085) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'linear'}
0.857143 (0.008690) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'poly'}
0.857143 (0.044070) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'rbf'}
0.678571 (0.261931) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'sigmoid'}
0.835714 (0.060085) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'linear'}
0.385714 (0.338475) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'poly'}
0.407143 (0.369107) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'rbf'}
0.150000 (0.001521) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'sigmoid'}
0.835714 (0.060085) with: {'C': 0.1, 'class_weight': 'balanced', 'gamma': 0.0001, 'kernel': 'linear'}
0.385714 (0.338475) with: 

In [611]:
svc = svm.SVC(gamma=0.001, class_weight='balanced', C = 1.7, kernel = 'linear')
#svc.fit(X_train, y_train) 

In [612]:
scores = cross_val_score(svc, X, y, cv=5, scoring='f1_weighted')
print(scores)
print("f1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
precision = cross_val_score(svc, X, y, cv=5, scoring='precision')
recall = cross_val_score(svc, X, y, cv=5, scoring='recall')
print(precision.mean())
print(recall.mean())

[0.86058758 0.90705839 0.9        0.92893773 0.97514793]
f1: 0.91 (+/- 0.08)
0.6825396825396826
0.8476190476190476


####SGDClassifier 

In [613]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
svc = SGDClassifier(max_iter= 1000)
parameters = [{'alpha': [0.01, 0.05, 0.001, 0.005], 'class_weight':['balanced']}]
                   
grid_search_item = GridSearchCV(svc,
                          param_grid = parameters,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_train, y_train)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

0.8714285714285714
{'alpha': 0.01, 'class_weight': 'balanced'}


In [614]:
clf = SGDClassifier(max_iter=1000, alpha=0.01, class_weight = 'balanced')
#clf.fit(X_train, y_train) 

SGDClassifier(alpha=0.01, average=False, class_weight='balanced', epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=1000, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [538]:
#y = clf.predict(X_test)


In [615]:
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_weighted')
print(scores)
print("f1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
precision = cross_val_score(clf, X, y, cv=5, scoring='precision')
recall = cross_val_score(clf, X, y, cv=5, scoring='recall')
print(precision.mean())
print(recall.mean())

[0.88599492 0.95121951 0.90562771 0.90694789 0.95157967]
f1: 0.92 (+/- 0.05)
0.6864285714285715
0.9095238095238095


###random forest

In [450]:
rf = RandomForestClassifier()

In [616]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)

parameters = [{'max_features':['auto','sqrt','log2'], 'class_weight':['balanced'], 
               'max_leaf_nodes':[10,50,100], 'max_depth':[2,5,10,20], 'n_estimators' : [50,100,200,300,400]}]
                   
grid_search_item = GridSearchCV(rf,
                          param_grid = parameters,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_train, y_train)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.9357142857142857
{'class_weight': 'balanced', 'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 50}
0.914286 (0.034306) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 50}
0.914286 (0.034306) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 100}
0.914286 (0.034306) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 200}
0.914286 (0.034306) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 300}
0.914286 (0.034306) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 10, 'n_estimators': 400}
0.914286 (0.034306) with: {'class_weight': 'balanced', 'max_depth': 2, 'max_features': 'auto', 'max_leaf_nodes': 50, 'n_estimators': 50}
0.914286 (0.034306) with: {'class_weight': '

In [628]:
rf = RandomForestClassifier(n_estimators=50, max_depth=5, class_weight = 'balanced',
                             random_state=50, max_leaf_nodes = 10, max_features = 'auto')

In [635]:
scores = cross_val_score(rf, X, y, cv=5, scoring='f1_weighted')
print(scores)
print("f1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
precision = cross_val_score(rf, X, y, cv=5, scoring='precision')
recall = cross_val_score(rf, X, y, cv=5, scoring='recall')
print(precision.mean())
print(recall.mean())

[0.92454523 1.         0.91408451 0.90694789 0.97338482]
f1: 0.94 (+/- 0.07)
0.8866666666666667
0.8095238095238095


In [638]:
bestModel = rf.fit(X_train, y_train) 
y_pred = bestModel.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[49,  0],
       [ 1, 10]])

In [640]:
X_test

array([[-4.07828929e-01,  2.27401788e+00,  2.90522313e+00,
        -3.03726857e-01,  2.49931837e+00,  2.42187631e+00,
         4.18945982e+00, -6.52945808e-02,  5.32230773e+00,
        -7.21303372e-01, -5.88839237e-01],
       [ 2.00893509e+00, -3.77898015e-01, -3.54494267e-01,
        -1.09262779e+00, -1.02084835e+00, -8.07292105e-01,
        -1.06707697e+00, -9.35888992e-01, -7.75262501e-01,
        -7.21303372e-01,  8.73969986e-01],
       [-4.07828929e-01, -3.77898015e-01, -3.54494267e-01,
        -3.03726857e-01, -3.16815005e-01, -1.61458421e-01,
        -1.57696104e-02, -6.52945808e-02,  9.58189607e-02,
         6.79285700e-01,  9.42286267e-01],
       [ 4.42569912e+00,  2.27401788e+00,  1.27536443e+00,
         1.27407500e+00,  5.31545175e+00,  1.13020895e+00,
        -1.57696104e-02,  3.41708306e+00, -7.75262501e-01,
        -7.21303372e-01,  1.23818831e+00],
       [ 2.00893509e+00,  2.27401788e+00,  1.27536443e+00,
         2.85187686e+00, -1.02084835e+00, -8.07292105e-01,
  

In [639]:
# dump object to pickles
filename = 'BestModel'
outfile = open(filename,'wb')

pickle.dump(bestModel,outfile)
outfile.close()

#Rule based classifier

In [332]:
rule = pd.read_csv('./LableResults/EvaluateSample3_fscore.csv')
rule[0:5]

,hash,text,count,cosineSim,label,manual
0,-4.240000e+18,VIRUS SPREADING LIKE WILDFIRE ON FB. ITS A TRO...,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.956189,0,1
1,7.570000e+18,They say true friends go long periods of time ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.924270,0,1
2,3.580000e+18,Going to church doesn't make you a holy person...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.936195,0,1
3,-1.990000e+18,From Lady Gaga's Bad Romance: /I want your psy...,"[4, 0, 0]",0.932406,0,1
4,2.910000e+18,The first woman was created from the rib of a ...,"[4, 1, 2, 0, 0, 0, 0, 0, 2]",0.976999,1,1


In [333]:
y_pred = rule['label']
y = rule['manual']

In [339]:
print('F1:', f1_score(y_pred, y, average= None))
cm = confusion_matrix(y_pred, y)
print('confusion Matrix:', cm)
print ('Recall:', recall_score(y_pred, y))
print ('Precision:', precision_score(y_pred, y))

F1: [0.98245614 0.89655172]
confusion Matrix: [[168   6]
 [  0  26]]
Recall: 1.0
Precision: 0.8125
